# 付録2： Numpy

Numpy は数値計算のためのパッケージで，配列を標準のリストではなく ndarray で扱う．
ある程度大きい配列や多次元の配列を扱う場合は Numpy の方が圧倒的に楽である．しかも速い．
ここでは主に標準のリストとの違いを説明する．

## 型

Numpy は数値計算を主眼に置いているので，基本的に数値データを扱う．数値データが扱えないわけではないが，時系列を見るための datetime が扱えない．時系列データを扱う場合は Pandas を使おう．

## 配列の演算

リストと ndarray の大きな違いの一つは，配列への演算の挙動である．ndarray の演算はほとんどの場合，ベクトル演算になる．
まず標準のリストに対して演算を行うとどうなるか試してみると，

In [2]:
a = [1, 2, 3, 4, 5]
b = [6, 7, 8, 9, 10]

print(a + b)
print(a * 3)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5]


「+」はリストの連結，「\* n」はリストを n回繰り返したものになる．一方で numpy では

In [24]:
import numpy as np

a = np.array([1, 2, 3, 4, 5])
b = np.array([6, 7, 8, 9, 10])

print(a + b)
print(a * 3)

[ 7  9 11 13 15]
[ 3  6  9 12 15]


となり，「+」は各要素ごとの和，「\* n」は各要素に n をかけたものになる．ベクトルの和，ベクトルとスカラの積に対応している．ちなみに配列同士のかけ算はリストではできないが，ndarray は要素ごとのかけ算となる．

In [25]:
print(a * b)

[ 6 14 24 36 50]


内積とか外積ではなく，アダマール積である．ベクトル積が必要な場合はそれぞれに対応した関数を使う．
注意することとして，ndarray で算術演算をする場合には ndarray のサイズが同じでなければエラーになる．

また，numpy 中の関数は同様に ndarray を引数にとり，その要素全てに関数を適用する，という動作をする (ユニバーサル関数)．

In [27]:
# 単項ufunc
print(np.sqrt(a))

# 二項ufunc
print(np.mod(b, a))

[1.         1.41421356 1.73205081 2.         2.23606798]
[0 1 2 1 0]


## スライシング

リストのところでも触れたように，ndarray ではスライスが多次元でも使えてものすごく便利である．

In [37]:
a = [[0, 1, 2, 3, 4], 
     ['a', 'b', 'c', 'd', 'e'], 
     ['A', 'B', 'C', 'D', 'E'], 
     ['p', 'q', 'r', 's', 't'], 
     ['P', 'Q', 'R', 'S', 'T']]
a_n = np.array(a)

print(a_n[2:5, 1:3])

[['B' 'C']
 ['q' 'r']
 ['Q' 'R']]


また，インデックスにTrue, False のリストを指定し，True の位置にあるものだけを抜き出すブールインデックス参照というのもできる．

In [39]:
a = np.array([1, 2, 3, 4, 5])
bidx = [True, False, False, True, False]

print(a[bidx])

[1 4]


この機能をそのまま使うことはあまりないが，比較演算子との組み合わせで非常に強力な機能を提供する．
上の演算のところで説明したように，ndarray に演算子を適用すると要素ごとに演算した結果を返す．これは算術演算だけでなく比較演算にも有効で，

In [40]:
a = np.array([1, 2, 3, 4, 5])
print(a > 3)

[False False False  True  True]


というように True/False のndarray を簡単に作ることができる．つまり，

In [41]:
print(a[a > 3])

[4 5]


のように「3より大きい要素を抜き出す」操作が書ける．`a > 3` がまず a>3 を満たすところだけが True の配列を作り，それをブールインデックスとしてスライシングしている．

もちろん多次元で通常の範囲指定と併用可能である．「2行目で 0より大きいもの」が欲しければ以下のように書けばよい．

In [51]:
a = np.random.randn(5, 5)
print(a)

print(a[1, a[1]>0])

[[-0.06326376 -0.77788805 -0.16812993  0.16772988  0.61138809]
 [ 0.2462664   0.20014139  1.38023302 -1.71574742 -1.04104565]
 [ 1.49008021  0.51533028 -0.43213365  0.58712364 -0.03589899]
 [-0.30803616 -0.00578722  0.18409996 -0.16635324  0.47237809]
 [ 0.13088245  0.05121704 -0.25228624 -1.38597913 -0.53716834]]
[0.2462664  0.20014139 1.38023302]


## スライス結果の扱い

ndarray のスライシングで注意しないといけないのは，スライスの結果の扱いである．ndarray のスライシングは，**スライスで指定された領域への参照 (ビュー)** を返す．つまり切り出された結果を要素にもつ新しい ndarray を作るわけではない．標準のリストでスライスが**切り出された要素をもつ新しいリスト**を返すのと異なるので，注意が必要．

まずは標準のリストの挙動を見ると，

In [5]:
a = [1, 2, 3, 4, 5]
b = a[2:4]
print(a)
print(b)

b[1] = 6
print(a)
print(b)

[1, 2, 3, 4, 5]
[3, 4]
[1, 2, 3, 4, 5]
[3, 6]


スライス `a[2:4]` は `[3, 4]` という**新しいリスト**を作るので，b に代入されるのはその新しいリストへの参照ということになる．つまり a と b は完全に別のリストなので，b の要素を更新しても a の要素は変わらない．同じようなことを ndarray でやってみると，

In [6]:
a = np.array([1, 2, 3, 4, 5])
b = a[2:4]
print(a)
print(b)

b[1] = 6
print(a)
print(b)

[1 2 3 4 5]
[3 4]
[1 2 3 6 5]
[3 6]


となる．`b[1] = 6` によって **a の要素も変わっている**．つまり b は a のある一部を指しているだけで，ndarray の実体は1つしかない．そのため，b を介して値を変えると，その参照先である a の要素も更新される．

明示的なコピー`copy()`や一部の操作 (ブールインデックスとか) を除き，ndarray はコピーが作られることはない．もともとndarray は巨大なデータ構造を扱うためのものなので，操作ごとにコピーを作っていてはメモリなどが大変なことになるから，らしい．

## 速度

Numpy は速い．どのぐらい速いか標準リストと同じ処理で比べてみると，

In [19]:
array_n = np.random.randn(1, 100000)
array_s = array_n.tolist()

In [20]:
%time for _ in range(10): array_n = array_n * 2

CPU times: user 1.37 ms, sys: 0 ns, total: 1.37 ms
Wall time: 933 µs


In [21]:
%time for _ in range(10): array_s = [x * 2 for x in array_s ]

CPU times: user 529 ms, sys: 346 ms, total: 875 ms
Wall time: 868 ms


ということで，Numpy でやった方が1000倍近く速い．